In [1]:
import nltk
#nltk.download()

In [2]:
import requests
import pandas as pd
import re, string, unicodedata
from pprint import pprint
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk import FreqDist
from textblob import TextBlob

#import contractions
#import inflect

In [3]:
violence_words = set(["asesinato", "asesino", "asesina", "matar", "mata", "mato", "apuñalar","apuñalo","pega","pego","apalizo","golpea","golpeo","agredio","agrede"])

In [4]:
print(violence_words)

{'mato', 'apalizo', 'pego', 'golpea', 'mata', 'asesino', 'agredio', 'asesina', 'apuñalo', 'apuñalar', 'pega', 'agrede', 'matar', 'asesinato', 'golpeo'}


In [5]:
stopWords = set(stopwords.words('spanish'))

In [6]:
print(stopWords)

{'otra', 'tengan', 'tuviese', 'nuestra', 'estáis', 'estaban', 'tuyo', 'hasta', 'vosostros', 'he', 'estabas', 'tuviera', 'él', 'poco', 'fuiste', 'seáis', 'tenía', 'uno', 'vuestro', 'tenéis', 'estar', 'vuestros', 'estarás', 'desde', 'hayan', 'muchos', 'nuestros', 'habremos', 'os', 'otros', 'entre', 'sí', 'otras', 'nuestro', 'y', 'mi', 'estuvieran', 'habidos', 'una', 'fuesen', 'tenido', 'tuviste', 'habían', 'serás', 'seremos', 'siente', 'eso', 'habría', 'esto', 'nosotras', 'hubierais', 'están', 'teníais', 'han', 'del', 'estuve', 'cuando', 'esos', 'haya', 'tuvieran', 'fueron', 'tienes', 'fueras', 'mucho', 'algunas', 'todo', 'esté', 'esas', 'tuviésemos', 'estuvo', 'fuésemos', 'e', 'sentida', 'ya', 'ante', 'tus', 'fueran', 'hay', 'hube', 'algo', 'tienen', 'o', 'sentid', 'fuéramos', 'somos', 'tendría', 'la', 'hayas', 'serías', 'sus', 'era', 'para', 'durante', 'tuviesen', 'estuviera', 'estado', 'vosostras', 'tendríamos', 'has', 'estaría', 'sea', 'éramos', 'también', 'hayamos', 'estas', 'estoy'

In [7]:
MADRID_URL = "https://www.madridiario.es/sucesos"

In [8]:
def get_soup(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser") #parser para que no se queje
    return soup

soup = get_soup(MADRID_URL)

In [36]:
#Get headers and its publication date
def get_headers(soup):
    headers = []
    date = []
    
    container = soup.find_all('div',{'class','n1 fueraNoticia'})
    for item in container:
        headers.append(item.find('h2',{'class','titulo'}).text)
        date.append(item.find('meta',itemprop='datePublished').get('content')[:10])

    if len(headers) == len(date):
        return pd.DataFrame({"headers": headers, "datePublication": date})
    else:
        return -1 #exit with error because there is news with date

        
dfnews_raw = get_headers(soup)
dfnews_raw.head(11)

,headers,datePublication
0,Agresión a un cerrajero cuando cambiaba la cer...,2019-06-27
1,Muere un mecánico al caerle un coche encima en...,2019-06-27
2,Vuelca con un coche robado en su huída de la G...,2019-06-27
3,Un incendio destruye una nave industrial en Hu...,2019-06-27
4,"Un niño de 8 años, grave tras un ahogamiento e...",2019-06-26
5,"Una anciana, crítica al quedar atrapada en su ...",2019-06-26
6,"La auxiliar de enfermería de Alcalá, culpable ...",2019-06-26
7,Desarticulada una organización de trata de tra...,2019-06-26
8,Detenidos por acordar el asesinato de un hombr...,2019-06-26
9,Libertad para el presunto pederasta que simula...,2019-06-26


In [45]:
# Get rid of lower-cases
def tolower (s):
    return s.lower()

dfnews_raw['headers'].apply(tolower)
dfnews_raw.head()

,headers,datePublication
0,Agresión a un cerrajero cuando cambiaba la cer...,2019-06-27
1,Muere un mecánico al caerle un coche encima en...,2019-06-27
2,Vuelca con un coche robado en su huída de la G...,2019-06-27
3,Un incendio destruye una nave industrial en Hu...,2019-06-27
4,"Un niño de 8 años, grave tras un ahogamiento e...",2019-06-26


In [34]:
# Get rid of accents
def clean_accent(s):
    return s.replace('á','a').replace('é','e').replace('í','i').replace('ó','o').replace('ú','u')

news_raw3['headers'] = dfnews_raw.headers.apply(clean_accent)
pprint(news_raw3[:6])

0    Agresion a un cerrajero cuando cambiaba la cer...
1    Muere un mecanico al caerle un coche encima en...
2    Vuelca con un coche robado en su huida de la G...
3    Un incendio destruye una nave industrial en Hu...
4    Un niño de 8 años, grave tras un ahogamiento e...
5    Una anciana, critica al quedar atrapada en su ...
Name: headers, dtype: object


In [ ]:
# Get rid of accents
def clean_accent(work_list):
    return [row.replace('á','a').replace('é','e').replace('í','i').replace('ó','o').replace('ú','u') for row in work_list]

news_raw2 = clean_accent(news_raw)
pprint(news_raw2[:6])

In [ ]:
#Get rid of commas
def clean_any_character(work_list, character):
    return [row.replace(character,'') for row in work_list]

news_raw3 = clean_any_character(news_raw2, ',')
pprint(news_raw3[:3])

In [ ]:
#Get rid of quotes
def clean_any_character(work_list, character):
    return [row.replace(character,'') for row in work_list]

news_raw4 = clean_any_character(news_raw3, '\"')
pprint(news_raw4[:4])

In [ ]:
#Get rid of '
def clean_any_character(work_list, character):
    return [row.replace(character,'') for row in work_list]

news_raw5 = clean_any_character(news_raw4, '\'')
pprint(news_raw5[:4])

In [ ]:
#Get rid of '
def clean_any_character(work_list, character):
    return [row.replace(character,'') for row in work_list]

news_raw6 = clean_any_character(news_raw5, ':')
pprint(news_raw6[:4])

In [ ]:
# Get each word / Tokenizar
def token(work_list):
    return [nltk.word_tokenize(row) for row in work_list]

token_news = token(news_raw6)
print(token_news[:6])

In [ ]:
def clean_stopwords(list_tokens, stp_words):
    [row.remove(item) for row in list_tokens for item in row if item in stp_words]
    return list_tokens
    
cleaned_data = clean_stopwords(token_news, stopWords)
pprint(cleaned_data[:6])

In [ ]:


#new = new + " enfado"
new = [' '.join(item) for item in cleaned_data]


for item in new:
    if TextBlob(item).detect_language() == 'es':
        print("1")
        analysis = TextBlob(str(item))
        item = analysis.translate(to='spanish')
        print(item)
    analysis = TextBlob(str(item))
    print(analysis.sentiment,"\n")




"""
new = ' '.join(cleaned_data[1])
print(new)
print(TextBlob(new).detect_language())
analysis = TextBlob(str(new))
print(analysis.sentiment,"\n")


new = ' '.join(cleaned_data[2])
print(new)
print(TextBlob(new).detect_language())
analysis = TextBlob(str(new))
print(analysis.sentiment,"\n")

"""
"""
new2 = analysis.translate(to='spanish')
analysis = TextBlob(str(new2))
print(analysis.sentiment)
print(analysis.tags)
#print(analysis.translate(to='spanish'))
print(dir(analysis))
"""


In [ ]:
def new_item(list_tokens):
    for index, row in enumerate(list_tokens):
        list_tokens.insert(index+1, [])
        list_tokens.insert(index+1, [])
    return list_tokens
 
print(new_item(cleaned_data))

In [ ]:
def violence_at_row(list_tokens,set_list):
    for row in list_tokens:
        for word in row:
            if word in violence_words:
                

In [ ]:
for word in cleaned_data:
    if 